# COMPSCI 761 22S2 Assignment 1

- Lecturer: Anna Trofimova
- School of Computer Science, The Univerity of Auckland
- Last update 29th of July at 18:00pm, 2022
$$
% macros
\newcommand{\indep}{\perp \!\!\!\perp}
$$

## Student:

Florian Suess, fsue534

### Submission
This interactive notebook contains the instructions to complete assignment 1; You should submit this notebook with the code and answers in one single file in .ipybn format with name assignment1.ipybn. **Write your name and UPI in the cell above** (to edit the markdown text double-click the cell).

There is a maximum file size cap of 5MB, so make sure your submission does not exceed this size. The submitted notebook should contain all your source code and answers. You can add new cells and use markdown text to organise and explain your implementation/answer.

Submit your files using CANVAS upload function.

The submission deadline is **19th of August at 11.59pm, 2022.**

### Plagiarism
This is an individual assignment. Remember that **all** work submitted for this assignment must be your own **individual** work and no code
sharing or copying is allowed. You may use code from the Internet only with suitable attribution
of the source in your program. **Do not use public code repositories as your code might be copied.** Keep in mind that sharing parts of assignment solutions is a form of plagiarism. All submitted assignments will be run through plagiarism detection software to detect similarities to other submissions. You should carefully read the UoA policy on academic integrity and plagiarism. 

### Description

This assignment consists of two problems presented in a puzzle game context. For the first problem, you will be implementing a path search problem along with common search algorithms and evaluating their properties. For the second problem, you will be exploring how to define and solve a constraint satisfaction problem. To implement the solutions, you have to use AIPython code that provides you with many implemented classes and examples of their use.  

When working with a Jupyter notebook, you can edit the \*.py files either in the Jupyter interface (in your browser) or with your favorite editor (e.g., PyCharm). Whenever you save a \*.py file, the notebook will reload their content directly.

The libraries that you can use (and need) in this assignment are  the following (run the cell below to import the libraries):

In [1]:
import sys
sys.path.insert(0, "./aipython")
import time
import numpy as np
from aipython.searchGeneric import *
from aipython.searchProblem import *
from aipython.cspProblem import *
from aipython.cspSearch import *

*Tip: If you placed the notebook correctly it should not give you any errors.*

*If you think that there are other libraries you might need then send an email to anna.trofimova@auckland.ac.nz to confirm.*

### Part 1 - Heuristics [5 marks]

This part of the assignment is based on a popular puzzle, Game of Fifteen, also known as 15-puzzle. The game consists of 15 numbered tiles on a four by four grid where one tile is missing. To solve the puzzle, the tiles must be moved so that they are ordered from 1 to 15. The goal state of the puzzle is defined as a multidimensional array of digits#, where 0 represents the missing tile, as follow:

In [2]:
goal = [[1, 2, 3, 4], 
        [5, 6, 7, 8], 
        [9, 10, 11, 12], 
        [13, 14, 15, 0]]

#### Task 1.1
Edit the code below to implement a search problem class representing the Game of Fifteen that uses heuristic based on Manhattan Distance (h1). The class must be implemented by extending the class Search\_problem. 

Manhattan Distance between two points $p1$ at $(x1, y1)$ and $p2$ at $(x2, y2)$ is $d_{M}(p1, p2) = |x1 - x2| + |y1 - y2|$

*Tip: If you have problems understanding how to overwrite methods take a look at the implementation of Search_problem_from_explicit_graph class in searchProblem.py*


In [67]:
import copy

"""
    PS: you have no idea how long it took me to realise that the nodes in this problem
    is the entire board configuration, *not each individual tile*... neighbours of this node being all
    board configurations that can be derived by 1 move (swapping the 0 tile with up/down/left/right if
    permissible).
"""

class GameFifteenProblem(Search_problem):
    ROWS=4
    COLUMNS=4
            
    def __init__(self, start: list[list[int]], goal: list[list[int]]):
        """Sets problem, will throw ValueError if board and/or goal given isn't to spec"""
        
        def must_be_valid_board(board: list[list[int]]) -> None:
            values: list[int] = []
            if len(board) != self.ROWS:
                raise ValueError(f"the fifteen game boards provided must have exactly {self.ROWS} rows")
            for row in board:
                if len(row) != self.COLUMNS:
                    raise ValueError(f"the fifteen game boards provided must have exactly {self.COLUMNS} columns")
                for value in row:
                    values.append(value)
            
            values.sort()
            if not np.array_equal(values, [i for i in range(16)]):
                raise ValueError(f"expect only 0,1,...,15 tiles as board input")

        must_be_valid_board(start)
        must_be_valid_board(goal)
        
        self.start_tiles: list[list[int]] = start
        self.goal_tiles: list[list[int]] = goal
        return

 

    def start_node(self):
        """Returns the start node, which in this case is the starting board configuration"""
        return self.start_tiles
    
    def is_goal(self, board: list[list[int]]) -> bool:
        """Returns True if the given board (list of tiles) is the goal, otherwise False"""
        for y, goal_row in enumerate(self.goal_tiles):
            for x, value in enumerate(goal_row):
                if board[y][x] != value:
                    return False
        
        return True
    
    def neighbors(self, board: list[list[int]]) -> bool:
        """Returns a list of the arcs for the neighbors of board, note the neighbours are
        at most four board configurations that are based on swaps with the 0 tile.
        
        for example: 
        return [Arc(node, to_neighbor_node1, cost1), Arc(node, to_neighbor_node2, cost2)]"""
        
        def swap(board: list[list[int]], zero: tuple[int, int], tile: tuple[int, int]):
            """Returns new board with zero and given tile swapped"""
            swapped_board = copy.deepcopy(board)
            
            swapped_board[zero[1]][zero[0]] = board[tile[1]][tile[0]]
            swapped_board[tile[1]][tile[0]] = 0
                
            return swapped_board
        
        neighbours: list[Arc] = []
        for y, row in enumerate(board):
            for x, value in enumerate(row):
                if value == 0:
                    """can only ever swap the 0 tile with it's immediate neighbours"""
                    zero_tile = (x, y)
                    
                    if y != 0:
                        zero_tile_swapped_up = swap(board, zero_tile, (x, y-1))
                        neighbours.append(Arc(board, zero_tile_swapped_up))
                        
                    if y + 1 != self.ROWS: 
                        zero_tile_swapped_down = swap(board, zero_tile, (x, y+1))
                        neighbours.append(Arc(board, zero_tile_swapped_down))
                        
                    if x != 0: 
                        zero_tile_swapped_left = swap(board, zero_tile, (x-1, y))
                        neighbours.append(Arc(board, zero_tile_swapped_left))
                        
                    if x + 1 != self.COLUMNS: 
                        zero_tile_swapped_right = swap(board, zero_tile, (x+1, y))
                        neighbours.append(Arc(board, zero_tile_swapped_right))
                    
        return neighbours
    
    def heuristic(self, board: list[list[int]]) -> int:
        """Returns the heuristic value of the node 
        based on the Manhattan distance"""
        
        heuristic: int = 0
            
        def get_goal_position(target: int) -> tuple[int, int]:
            for y, row in enumerate(self.goal_tiles):
                for x, value in enumerate(row):
                    if value == target:
                        return (x, y)

        for y, row in enumerate(board):
            for x, value in enumerate(row):
                goal_position = get_goal_position(value)
                heuristic += ( abs(x-goal_position[0]) + abs(y-goal_position[1]) )
                
        return heuristic
    
    

To validate the correctness of the problem class use the A* searcher algorithm (from searchGeneric.py) to find a solution. 

*Tip: The cost of the solution should be 9.*

In [83]:
start = [[1, 2, 3, 4], 
         [9, 5, 6, 7], 
         [10, 11, 8, 0], 
         [13, 14, 15, 12]]

puzzle = GameFifteenProblem(start, goal)
searcher = AStarSearcher(puzzle)
solution = searcher.search()
print('A* Cost: ',  solution.cost)

45 paths have been expanded and 89 paths remain in the frontier
A* Cost:  9


#### Task 1.2 
Implement search problem classes representing the Game of Fifteen that use heuristics based on Euclidean Distance (h2) and the number of the inversions of the permutation (h3). The classes must be implemented by extending the class GameFifteenProblem.

Euclidean distance between two points $p1$ at $(x1, y1)$ and $p2$ at $(x2, y2)$ is $d_{E}(p1, p2) =  \sqrt{(x1-x2)^{2}+ (y1-y2)^{2}}$

An inversion of a permutation $(t_{1},t_{2},...,t_{n})$ of the elements in a finite n-element set of positive integers $A=\{1,2,...,n\}$ is the pair $(t_{j},t_{k})$, where $j<k$ and $t_{j}>t_{k}$.

$\sum_{j=1}^{N} \sum_{k=j}^{N}    \{t_{j} > t_{k}: 1, otherwise: 0\}$, where $N$ is the total number of elements and $t_{i}$ is the value of i-th element.

In [19]:
class GameFifteenProblemEuclidean(GameFifteenProblem):
    def __init__(self, start: list[list[int]], goal: list[list[int]]):
        (super().__init__(start, goal))

    def heuristic(self, board: list[list[int]]):
        heuristic: int = 0
            
        def get_goal_position(target: int) -> tuple[int, int]:
            for y, row in enumerate(self.goal_tiles):
                for x, value in enumerate(row):
                    if value == target:
                        return (x, y)

        for y, row in enumerate(board):
            for x, value in enumerate(row):
                goal_position = get_goal_position(value)
                
                heuristic += ( (x-goal_position[0])**2 + (y-goal_position[1])**2 )**(1/2)
                
        return heuristic

In [34]:
class GameFifteenProblemInversions(GameFifteenProblem):
    def __init__(self, start, goal):
        (super().__init__(start, goal))

    def heuristic(self, board: list[list[int]]):

        flattened_board: list[int] = []
        for row in board:
            for value in row:
                flattened_board.append(value)
        
        heuristic: int = 0
        for j in range(16):
            for k in range(j, 16):
                left, right = flattened_board[j], flattened_board[k]
                
                """ ignore zero tile completely """
                if left == 0 or right == 0:
                    continue

                if left > right:
                    heuristic += 1

        return heuristic

#### Task 1.3
Run A* Search algorithm with every heuristic for the following three start states:

In [35]:
# optimal path cost: 14
start14 = [[1, 2, 8, 3],
           [5, 6, 7, 4],
           [9, 15, 14, 11],
           [13, 10, 12, 0]]

# optimal path cost: 17
start17 = [[1, 3, 6, 4],
           [5, 2, 8, 14],
           [9, 15, 7, 0],
           [13, 10, 12, 11]]

# optimal path cost: 23
start23 = [[1, 3, 6, 4],
           [5, 8, 15, 14],
           [9, 2, 7, 0],
           [13, 10, 12, 11]]

for start in [start14, start17, start23]:
    print('=========')
    print('Starting Position: ', start)
    print('Manhattan: ', end='')
    manhattan_based_searcher = AStarSearcher(GameFifteenProblem(start, goal))
    manhattan_based_searcher.search()
    
    print('Euclidean: ', end='')
    euclidean_based_searcher = AStarSearcher(GameFifteenProblemEuclidean(start, goal))
    euclidean_based_searcher.search()
    
    print('Inversions: ', end='')
    inversions_based_searcher = AStarSearcher(GameFifteenProblemInversions(start, goal))
    inversions_based_searcher.search()
    print()

Starting Position:  [[1, 2, 8, 3], [5, 6, 7, 4], [9, 15, 14, 11], [13, 10, 12, 0]]
Manhattan: 384 paths have been expanded and 682 paths remain in the frontier
Euclidean: 617 paths have been expanded and 1122 paths remain in the frontier
Inversions: 30 paths have been expanded and 62 paths remain in the frontier

Starting Position:  [[1, 3, 6, 4], [5, 2, 8, 14], [9, 15, 7, 0], [13, 10, 12, 11]]
Manhattan: 533 paths have been expanded and 967 paths remain in the frontier
Euclidean: 431 paths have been expanded and 778 paths remain in the frontier
Inversions: 1029 paths have been expanded and 2422 paths remain in the frontier

Starting Position:  [[1, 3, 6, 4], [5, 8, 15, 14], [9, 2, 7, 0], [13, 10, 12, 11]]
Manhattan: 14380 paths have been expanded and 25805 paths remain in the frontier
Euclidean: 24760 paths have been expanded and 46179 paths remain in the frontier
Inversions: 87774 paths have been expanded and 192844 paths remain in the frontier



In each case record in the tables below the heuristic values for the start states, the number of the expanded nodes, and the costs of the solutions.

| Heuristic | h-value: start14 | h-value: start17 | h-value: start23 |
|-----------|------------------|------------------|------------------|
|Manhattan  | #                | #                | #                | 
|Euclidean  | #                | #                | #                | 
|Inversions | #                | #                | #                | 

| Heuristic | expanded: start14 | expanded: start17 | expanded: start23 |
|-----------|-------------------|-------------------|-------------------|
|Manhattan  | #                 | #                 | #                 | 
|Euclidean  | #                 | #                 | #                 |  
|Inversions | #                 | #                 | #                 |  

| Heuristic | path cost: start14 | path cost: start17 | path cost: start23 |
|-----------|--------------------|--------------------|--------------------|
|Manhattan  | #                  | #                  | #                  | 
|Euclidean  | #                  | #                  | #                  |
|Inversions | #                  | #                  | #                  |

Comment on the performance of the A* search algorithm with each heuristic based on the results in the tables. Explain which heuristic is better/worse and why.

### Part 2:  Search algorithms [7 marks]

*Tip: If you have problems understanding how to overwrite methods take a look at the implementation of AStarSearcher and  Searcher classes in searchGeneric.py*

*Tip: To initialize the frontier think of the type of structure used by the algorithm to store generated nodes. If it uses a priority queue use FrontierPQ from serachGeneric.py*

#### Task 2.1
Implement a class that performs Breadth First Search by extending class Searcher.

In [37]:
"""
All we change from the `Searcher` class is how we select from the frontier, so leaving all methods as is
and swapping from a stack (`pop()`) to a queue (`pop(0)`).

Otherwise straight from the `aipython/searchGeneric.py` implementation
"""

class BreadthFirstSearcher(Searcher):
    def __init__(self,  problem) -> None:
        super().__init__(problem)

    def search(self):
        while not self.empty_frontier():
            path = self.frontier.pop(0)
            self.display(2, "Expanding:",path,"(cost:",path.cost,")")
            self.num_expanded += 1
            if self.problem.is_goal(path.end()):
                self.display(1, self.num_expanded, "paths have been expanded and", 
                             len(self.frontier), "paths remain in the frontier")
                self.solution = path
                return path
            else:
                neighs = self.problem.neighbors(path.end())
                self.display(3,"Neighbors are", neighs)
                for arc in reversed(list(neighs)):
                    self.add_to_frontier(Path(path,arc))
                self.display(3,"Frontier:",self.frontier)
        self.display(1,"No (more) solutions. Total of",self.num_expanded,"paths expanded.")

#### Task 2.2

Implement a class that performs Iterative Deepening Search by extending class Searcher.

In [71]:
class IterativeDeepeningSearcher(Searcher):
    def __init__(self, problem):
        self.iterative_depth = 0
        self.problem = problem
        self.initialize_frontier()
        self.num_expanded = 0
        self.add_to_frontier(Path(problem.start_node()), 0)

    def add_to_frontier(self, path, depth):
        """ adds the path and it's current depth to the frontier """
        self.frontier.append((path, depth))
    
    def search(self):
        forced_deadends = 0 
        """ 
            number of popped nodes in the frontier disregarded due to iterative depth, used
            to decide wether to bump the iterative depth of the search, or wave hands re: no solution found
        """
        
        while not self.empty_frontier():
            path, current_depth = self.frontier.pop()
            self.display(2, "Expanding:",path,"depth level: ", current_depth,"(cost:",path.cost,")")
            self.num_expanded += 1
            if current_depth > self.iterative_depth:
                self.display(2, "Cannot continue, hit iterative depth level")
                forced_deadends += 1
                continue
            elif self.problem.is_goal(path.end()):
                self.display(1, self.num_expanded, "paths have been expanded and", 
                             len(self.frontier), "paths remain in the frontier")
                self.solution = path
                return path
            else:
                neighs = self.problem.neighbors(path.end())
                self.display(3,"Neighbors are", neighs)
                for arc in reversed(list(neighs)):
                    self.add_to_frontier(Path(path,arc), current_depth+1)
                self.display(3,"Frontier:",self.frontier)

        if forced_deadends > 0:
            self.iterative_depth += 1
            self.display(1,"Found",forced_deadends,"forced dead ends",
                         "increasing iterative depth to", self.iterative_depth)
            self.add_to_frontier(Path(self.problem.start_node()), 0)
            return self.search()
        else:                             
            self.display(1,"No (more) solutions. Total of",self.num_expanded,"paths expanded.")

#### Task 2.3
Implement a class that performs Iterative Deepening A* Search by extending class Searcher.

In [117]:
"""
We see an A* search algorithm implementation in `searchGeneric.py`, we use that
implementation, then borrow what we have done above re: search method and adapt the bound to be
instead based on the lowest "f value" removed from the frontier queue during each iteration.
"""

class IterativeDeepeningAStarSearcher(Searcher):
    def __init__(self, problem):
        self.problem = problem
        self.initialize_frontier()
        self.num_expanded = 0

        initial = Path(problem.start_node())
        self.iterative_f_value_depth = initial.cost + self.problem.heuristic(initial.end())
        self.add_to_frontier(initial, self.iterative_f_value_depth)

    def initialize_frontier(self):
        self.frontier = FrontierPQ()

    def empty_frontier(self):
        return self.frontier.empty()

    def add_to_frontier(self,path,f_value):
        self.frontier.add(path, f_value)
    
    @visualize
    def search(self):
        forced_deadends = 0 
        """ 
            number of popped nodes in the frontier disregarded due to iterative depth, used
            to decide wether to bump the iterative depth of the search, or wave hands re: no solution found
        """
        
        min_forced_deadend_f_value = None
        """ 
            minimum "f value" of popped frontier paths due to meeting the forced iterative depth, used
            to decide next iterations iterative f value depth.
        """
        def set_min_forced_deadend_f_value(candidate_f_value: int) -> None:
            nonlocal min_forced_deadend_f_value
            if min_forced_deadend_f_value is None:
                min_forced_deadend_f_value = candidate_f_value
                return
            min_forced_deadend_f_value = min(min_forced_deadend_f_value, candidate_f_value)
        
        while not self.empty_frontier():
            path = self.frontier.pop()
            f_value = path.cost + self.problem.heuristic(path.end())
            
            self.display(2, "Expanding:",path,"f value level: ", min_forced_deadend_f_value,"(cost:",path.cost,")")
            self.num_expanded += 1
            if f_value > self.iterative_f_value_depth:
                self.display(2, "Cannot continue, hit iterative f value depth level")
                forced_deadends += 1
                set_min_forced_deadend_f_value(f_value)
                continue
            elif self.problem.is_goal(path.end()):
                self.display(1, self.num_expanded, "paths have been expanded and", 
                             len(self.frontier), "paths remain in the frontier")
                self.solution = path
                return path
            else:
                neighs = self.problem.neighbors(path.end())
                self.display(3,"Neighbors are", neighs)
                for arc in reversed(list(neighs)):
                    self.add_to_frontier(Path(path,arc), f_value)
                self.display(3,"Frontier:",self.frontier)

        if forced_deadends > 0:
            self.iterative_f_value_depth = min_forced_deadend_f_value
            self.display(1,"Found",forced_deadends,"forced dead ends",
                         "increasing iterative f value depth to", self.iterative_f_value_depth)
            self.add_to_frontier(Path(self.problem.start_node()), min_forced_deadend_f_value)
            return self.search()
        else:                             
            self.display(1,"No (more) solutions. Total of",self.num_expanded,"paths expanded.")

#### Task 2.4
Implement a class that performs Uniform Cost Search by extending class Searcher.

In [126]:
class UniformCostSearcher(Searcher):
    """ Initializes the forontier """
    def initialize_frontier(self):
        self.frontier = FrontierPQ()

    """ Returns True if there are no more nodes to expand """
    def empty_frontier(self):
        return self.frontier.empty()

    """ Adds the path to the forontier """
    def add_to_frontier(self, path):
        self.frontier.add(path, path.cost + self.problem.heuristic(path.end()))

#### Task 2.5

Run Breadth First Search (BFS), Iterative Deepenining Search (IDS), Iterative Deepening A* Search (IDA\*S), and Uniform Cost Search (UCS) algorithms on each of the 5 start states:

In [133]:
# optimal path cost: 10
start10 = [[2, 3, 7, 4],
           [1, 6, 11, 8],
           [5, 10, 0, 12],
           [9, 13, 14, 15]]

# optimal path cost: 24
start24 = [[2, 7, 11, 4],
           [6, 3, 12, 0],
           [1, 5, 15, 8],
           [9, 10, 13, 14]]

# optimal path cost: 30
start30 = [[2, 7, 11, 4],
           [6, 3, 12, 0],
           [1, 5, 15, 14],
           [9, 10, 8, 13]]

# optimal path cost: 36
start36 = [[7, 11, 12, 4],
           [2, 3, 14, 1],
           [6, 5, 13, 8],
           [9, 10, 15, 0]]

# optimal path cost: 41
start41 = [[7, 11, 12, 4],
           [2, 3, 8, 14],
           [10, 0, 5, 1],
           [6, 9, 13, 15]]

class BoundedBreadthFirstSearcher(BreadthFirstSearcher):        
    def add_to_frontier(self,path):
        self.frontier.append(path)
        if len(self.frontier) > 1_000_000:
            raise MemoryError("maximum frontier size reached")
    

for start in [start10, start24, start30, start36, start41]:
    print('=========')
    print('Starting Position: ', start)
    
    print('---------')
    print('BFS:')
    puzzle = GameFifteenProblem(start, goal)
    bfs_searcher = BoundedBreadthFirstSearcher(puzzle)
    bfs_solution = bfs_searcher.search()
    
    print('---------')
    print('IDA:')
    puzzle = GameFifteenProblem(start, goal)
    ida_searcher = IterativeDeepeningSearcher(puzzle)
    ida_solution = ida_searcher.search()
    
    print('---------')
    print('IDA A Star:')
    puzzle = GameFifteenProblem(start, goal)
    ida_star_searcher = IterativeDeepeningAStarSearcher(puzzle)
    ida_star_solution = ida_star_searcher.search()
    
    print('---------')
    print('UCS:')
    puzzle = GameFifteenProblem(start, goal)
    ucs_searcher = UniformCostSearcher(puzzle)
    ucs_solution = ucs_searcher.search()

Starting Position:  [[2, 3, 7, 4], [1, 6, 11, 8], [5, 10, 0, 12], [9, 13, 14, 15]]
---------
BFS:
243079 paths have been expanded and 543549 paths remain in the frontier
---------
IDA:
Found 4 forced dead ends increasing iterative depth to 1
Found 14 forced dead ends increasing iterative depth to 2
Found 46 forced dead ends increasing iterative depth to 3
Found 150 forced dead ends increasing iterative depth to 4
Found 486 forced dead ends increasing iterative depth to 5
Found 1574 forced dead ends increasing iterative depth to 6
Found 5094 forced dead ends increasing iterative depth to 7
Found 16486 forced dead ends increasing iterative depth to 8
Found 53350 forced dead ends increasing iterative depth to 9
Found 172646 forced dead ends increasing iterative depth to 10
393280 paths have been expanded and 11 paths remain in the frontier
---------
IDA A Star:
Found 4 forced dead ends increasing iterative f value depth to 13
Found 12 forced dead ends increasing iterative f value depth to

KeyboardInterrupt: 

In each case, record in the table below the number of nodes generated during the search. If the algorithm runs out of memory or the number of nodes in the frontier exceeds 1 million items, just write “Mem” in your table. If the code runs for five minutes without producing output, terminate the process and write “Time” in your table.

*Tip: To edit the table double click the cell below.*

| Algorithm | 10      | 24      | 30      | 36      | 41      |
|-----------|---------|---------|---------|---------|---------|
|BFS        | #       | #       | #       | #       | #       |
|A*         | #       | #       | #       | #       | #       |
|IDS        | #       | #       | #       | #       | #       |
|IDA\*S     | #       | #       | #       | #       | #       |
|UCS        | #       | #       | #       | #       | #       |

Discuss the time and space efficiency of these four algorithms, comment on the results in the table.

### Part 3: Deceptive Starting States [3 marks]

#### Task 3.1 

Run IDA* on the starting states below and report the number of the expanded nodes.

In [26]:
start37_1 = [[7, 11, 12, 4],
             [2, 3, 14, 1],
             [6, 5, 13, 0],
             [9, 10, 15, 8]]

start37_2 = [[7, 11, 12, 4],
             [2, 3, 8, 14],
             [6, 0, 1, 15],
             [9, 5, 10, 13]]
# your code

#### Task 3.2

Explain why there is a difference between the number of the expanded nodes for these starting states if their costs of the optimal paths are the same.

### Part 4:  Constraint Satisfaction Problem [5 marks]

This part of the assignment is based on another puzzle called Sudoku. The game consists of 9x9 grid with a few cells filled in with digits. The objective of the game is to fill the remaining cells so that each column, each row, and each of the nine 3×3 sub-grids contain all of the digits from 1 to 9.

The start state of the puzzle is defined as a multidimensional array of numbers, where 0 represents empty cells, for example:

In [18]:
grid = [[9, 5, 0, 8, 2, 7, 3, 0, 0],
        [0, 8, 0, 1, 4, 0, 0, 5, 0],
        [0, 1, 0, 5, 9, 0, 0, 0, 0],
        [8, 3, 0, 0, 0, 0, 0, 7, 5],
        [1, 6, 9, 7, 5, 2, 4, 3, 0],
        [0, 7, 0, 0, 8, 0, 0, 6, 0],
        [0, 9, 1, 0, 6, 0, 8, 4, 0],
        [7, 0, 8, 0, 3, 1, 0, 0, 6],
        [6, 2, 0, 4, 7, 8, 0, 9, 0]]

*Tip: If you have problems with understanding how to implement constrains take a look at cspExamples.py and cspExamplesQueens.py*

#### Task 4.1

Write the code that defines constraint(s) and a function grid_to_csp that returns a CSP for a Sudoku puzzle:

In [ ]:
# define constraint function(s)

# function that returns a csp
def grid_to_csp(grid):
    domains = {}
    constraints = []
    return CSP(domains, constraints)

# csp
csp = grid_to_csp(grid)

#### Task 4.2

Write the code that solves Sudoku csp using a search algorithms of your choice, print the solution.

In [17]:
# your code


#### Task 4.3

Descibe what do nodes represent in the search tree. Explain your choice of the search algorithm and compare it with any other search algorithm implemented in this assignment. 